# Measuring Cell Deformation

## Installs, imports, and uploads

In [1]:
# #@title Package Installations

# !pip install numpy opencv-python-headless pandas matplotlib stardist ipywidgets plotly --quiet



In [2]:
%matplotlib widget

In [3]:
# #@title File Upload

# from google.colab import files
# uploaded = files.upload()

# Just in case we're using this notebook from Colab

In [4]:
#@title Imports

import copy

import cv2
import numpy as np
from stardist.models import StarDist2D
from skimage import io, color
import pandas as pd

import matplotlib.pyplot as plt

from scipy.spatial.distance import cdist

from ipywidgets import Button, HBox, interact, IntSlider, Output
from IPython.display import display, clear_output
import ipywidgets as widgets

from skimage.morphology import opening, disk, closing
from skimage.filters import gaussian
from skimage.color import label2rgb






2024-03-20 12:10:52.569179: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 12:10:52.571758: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 12:10:52.605480: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 12:10:53.604081: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Loading Video and Frames

In [5]:
#@title Load Model and Data

# Load the pre-trained StarDist model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

# Initialize a list to store segmentation results
segmentation_results = []

# Load the video
video_path = 'Movie 11 - Fig. S31_actual 32.avi'
cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Preload frames (we could load a subset if the video was very large)
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame_rgb[:, :, 1])  # Store only the green channel

cap.release()

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [7]:
#@title Rendering the Original Video

current_frame_idx = 0  # Track the current frame index

plot_output = widgets.Output()  # Dedicated output widget for the plot

def plot_frame(frame_idx):
    with plot_output:
        clear_output(wait=True)  # Clear the previous plot
        plt.figure(figsize=(8, 8))
        plt.imshow(frames[frame_idx], cmap='Greens')
        plt.title(f"Frame {frame_idx}")
        plt.axis('off')
        plt.show()

def on_previous_clicked(b):
    global current_frame_idx
    if current_frame_idx > 0:
        current_frame_idx -= 1
        slider.value = current_frame_idx  # This triggers plot update

def on_next_clicked(b):
    global current_frame_idx
    if current_frame_idx < len(frames) - 1:
        current_frame_idx += 1
        slider.value = current_frame_idx  # This triggers plot update

previous_button = Button(description="Previous")
next_button = Button(description="Next")

previous_button.on_click(on_previous_clicked)
next_button.on_click(on_next_clicked)

slider = IntSlider(min=0, max=len(frames)-1, step=1, value=0)

def on_slider_change(change):
    global current_frame_idx
    current_frame_idx = change['new']
    plot_frame(current_frame_idx)  # Update plot directly for smoother slider action

slider.observe(on_slider_change, names='value')

controls = HBox([previous_button, next_button, slider])  # Group the controls

# Display the controls and the initial plot separately
display(controls)
display(plot_output)  # This will hold the plot
plot_frame(current_frame_idx)  # Initial plot


Output()

## Preprocessing (denoising)

In [8]:

# Apply morphological opening to each frame to remove noise
# Disk-shaped structuring element with a radius of 3 pixels
selem = disk(3)

# We will store the processed frames in a separate list
processed_frames = copy.deepcopy(frames)

# Apply Gaussian blur to each opened frame (sigma controls the amount of blurring)
processed_frames = [gaussian(frame, sigma=2) for frame in processed_frames]

# Apply morphological opening to each frame to remove noise
processed_frames = [opening(frame, selem) for frame in processed_frames]

# processed_frames = [opening(frame, selem) for frame in processed_frames]
# processed_frames = [closing(frame, selem) for frame in processed_frames]
# processed_frames = [opening(frame, selem) for frame in processed_frames]
# processed_frames = [closing(frame, selem) for frame in processed_frames]


In [10]:
#@title Rendering the Processed Video

current_frame_idx = 0  # Track the current frame index

plot_output = widgets.Output()  # Dedicated output widget for the plot

def plot_frame(frame_idx):
    with plot_output:
        clear_output(wait=True)  # Clear the previous plot
        plt.figure(figsize=(8, 8))
        plt.imshow(processed_frames[frame_idx], cmap='Greens')
        plt.title(f"Frame {frame_idx}")
        plt.axis('off')
        plt.show()

def on_previous_clicked(b):
    global current_frame_idx
    if current_frame_idx > 0:
        current_frame_idx -= 1
        slider.value = current_frame_idx  # This triggers plot update

def on_next_clicked(b):
    global current_frame_idx
    if current_frame_idx < len(processed_frames) - 1:
        current_frame_idx += 1
        slider.value = current_frame_idx  # This triggers plot update

previous_button = Button(description="Previous")
next_button = Button(description="Next")

previous_button.on_click(on_previous_clicked)
next_button.on_click(on_next_clicked)

slider = IntSlider(min=0, max=len(processed_frames)-1, step=1, value=0)

def on_slider_change(change):
    global current_frame_idx
    current_frame_idx = change['new']
    plot_frame(current_frame_idx)  # Update plot directly for smoother slider action

slider.observe(on_slider_change, names='value')

controls = HBox([previous_button, next_button, slider])  # Group the controls

# Display the controls and the initial plot separately
display(controls)
display(plot_output)  # This will hold the plot
plot_frame(current_frame_idx)  # Initial plot


Output()

## Instance Segmentation

In [11]:

# Perform segmentation on each opened frame
segmentation_labels = [model.predict_instances(frame)[0] for frame in processed_frames]


2024-03-20 12:15:39.638179: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 134217728 exceeds 10% of free system memory.
2024-03-20 12:15:39.799496: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 134217728 exceeds 10% of free system memory.
2024-03-20 12:15:40.183605: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.
2024-03-20 12:15:40.265922: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 134217728 exceeds 10% of free system memory.
2024-03-20 12:15:42.499210: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 134217728 exceeds 10% of free system memory.


In [12]:
#@title Rendering the Segmented Video

segmentation_output = Output()  # Dedicated output widget for segmentation plots

def plot_segmentation(frame_idx):
    with segmentation_output:
        clear_output(wait=True)
        label_overlay = label2rgb(segmentation_labels[frame_idx], image=processed_frames[frame_idx], bg_label=0, alpha=0.4)
        
        plt.figure(figsize=(10, 10))
        plt.imshow(label_overlay)
        plt.title(f"Segmented Frame {frame_idx} (with Gaussian Blur)")
        plt.axis('off')
        plt.show()

def on_previous_clicked(b):
    global current_frame_idx
    if current_frame_idx > 0:
        current_frame_idx -= 1
        slider.value = current_frame_idx  # This triggers plot update

def on_next_clicked(b):
    global current_frame_idx
    if current_frame_idx < len(processed_frames) - 1:
        current_frame_idx += 1
        slider.value = current_frame_idx  # This triggers plot update

previous_button = Button(description="Previous")
next_button = Button(description="Next")

previous_button.on_click(on_previous_clicked)
next_button.on_click(on_next_clicked)

slider = IntSlider(min=0, max=len(processed_frames)-1, step=1, value=0)

def on_slider_change(change):
    global current_frame_idx
    current_frame_idx = change['new']
    plot_segmentation(current_frame_idx)  # Directly update the plot for segmentation results

slider.observe(on_slider_change, names='value')

controls = HBox([previous_button, next_button, slider])

# Display the controls and the initial segmentation plot separately
display(controls)
display(segmentation_output)  # This will hold the segmentation plot
plot_segmentation(current_frame_idx)  # Initial plot


Output()

In [11]:
%matplotlib notebook


In [14]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [12]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, IntSlider, Output, VBox
from IPython.display import display, clear_output
import ipywidgets as widgets


segmentation_output = Output()  # Dedicated output widget for segmentation plots

def onclick(event, frame_idx):
    # Check if click was inside the axes
    if event.inaxes is not None:
        x, y = int(event.xdata), int(event.ydata)
        label = segmentation_labels[frame_idx][y, x]
        print(f"Clicked on label: {label}")

def plot_segmentation(frame_idx):
    with segmentation_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(10, 6))
        label_overlay = label2rgb(segmentation_labels[frame_idx], image=processed_frames[frame_idx], bg_label=0, alpha=0.4)
        
        ax.imshow(label_overlay)
        ax.set_title(f"Segmented Frame {frame_idx} (Click on an object to identify)")
        ax.axis('off')
        
        # Connect the click event
        cid = fig.canvas.mpl_connect('button_press_event', lambda event: onclick(event, frame_idx))
        
        plt.show()


def on_previous_clicked(b):
    global current_frame_idx
    if current_frame_idx > 0:
        current_frame_idx -= 1
        slider.value = current_frame_idx  # This triggers plot update

def on_next_clicked(b):
    global current_frame_idx
    if current_frame_idx < len(processed_frames) - 1:
        current_frame_idx += 1
        slider.value = current_frame_idx  # This triggers plot update

previous_button = Button(description="Previous")
next_button = Button(description="Next")

previous_button.on_click(on_previous_clicked)
next_button.on_click(on_next_clicked)

slider = IntSlider(min=0, max=len(processed_frames)-1, step=1, value=0)

def on_slider_change(change):
    global current_frame_idx
    current_frame_idx = change['new']
    plot_segmentation(current_frame_idx)  # Directly update the plot for segmentation results

slider.observe(on_slider_change, names='value')

controls = HBox([previous_button, next_button, slider])

# Display the controls and the initial segmentation plot separately
display(controls)
display(segmentation_output)  # This will hold the segmentation plot
plot_segmentation(current_frame_idx)  # Initial plot


Output()

In [13]:
from skimage.measure import regionprops
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, IntSlider, Output, VBox
from IPython.display import display, clear_output
import ipywidgets as widgets

# Assuming 'segmentation_labels' and 'processed_frames' are defined

segmentation_output = Output()

def plot_segmentation_with_labels(frame_idx):
    with segmentation_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(10, 6))
        label_overlay = label2rgb(segmentation_labels[frame_idx], image=processed_frames[frame_idx], bg_label=0, alpha=0.4)
        
        ax.imshow(label_overlay)
        ax.set_title(f"Segmented Frame {frame_idx}")
        ax.axis('off')
        
        # Iterate over all labeled regions
        for region in regionprops(segmentation_labels[frame_idx]):
            # Draw rectangle around segmented coins
            minr, minc, maxr, maxc = region.bbox
            bbox_center = (minc + maxc) / 2, (minr + maxr) / 2
            ax.text(bbox_center[0], bbox_center[1], str(region.label), color='red', fontsize=8, ha='center', va='center')
        
        plt.show()

def on_previous_clicked(b):
    global current_frame_idx
    if current_frame_idx > 0:
        current_frame_idx -= 1
        slider.value = current_frame_idx  # This triggers plot update

def on_next_clicked(b):
    global current_frame_idx
    if current_frame_idx < len(processed_frames) - 1:
        current_frame_idx += 1
        slider.value = current_frame_idx  # This triggers plot update

previous_button = Button(description="Previous")
next_button = Button(description="Next")

previous_button.on_click(on_previous_clicked)
next_button.on_click(on_next_clicked)

slider = IntSlider(min=0, max=len(processed_frames)-1, step=1, value=0)

def on_slider_change(change):
    global current_frame_idx
    current_frame_idx = change['new']
    plot_segmentation(current_frame_idx)  # Directly update the plot for segmentation results

slider.observe(on_slider_change, names='value')

controls = HBox([previous_button, next_button, slider])

# Display the controls and the initial segmentation plot separately
display(controls)
display(segmentation_output)  # This will hold the segmentation plot
plot_segmentation(current_frame_idx)  # Initial plot

Output()

## Object Tracking

In [ ]:
#@title Object Tracking

# Function to calculate the centroids of labeled objects
def calculate_centroids(labels):
    unique_labels = np.unique(labels)
    centroids = []
    for label in unique_labels:
        if label == 0:
            continue  # Skip background
        ys, xs = np.where(labels == label)
        centroids.append((np.mean(xs), np.mean(ys)))
    return np.array(centroids)

# Initialize DataFrame to store results
df = pd.DataFrame(columns=['Frame', 'Label', 'Area', 'Centroid_X', 'Centroid_Y'])

for i, labels in enumerate(segmentation_results):
    centroids = calculate_centroids(labels)
    previous_centroid = calculate_centroids(segmentation_results[i-1])[selected_label-1] if i > 0 else calculate_centroids(first_frame_segmentation)[selected_label-1]
    distances = cdist([previous_centroid], centroids)
    selected_label = np.argmin(distances) + 1  # +1 to adjust for background being skipped
    selected_area = np.sum(labels == selected_label)
    selected_centroid = centroids[selected_label-1]
    df = df.append({'Frame': i, 'Label': selected_label, 'Area': selected_area, 'Centroid_X': selected_centroid[0], 'Centroid_Y': selected_centroid[1]}, ignore_index=True)
